In [3]:
!pip install GitPython

  Using cached GitPython-3.1.32-py3-none-any.whl (188 kB)
  Using cached gitdb-4.0.10-py3-none-any.whl (62 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)

[notice] A new release of pip is available: 23.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import requests
import pandas as pd

In [8]:
df = pd.read_csv("/Users/pnair/Documents/23Shades-master/data/sampled_label_dataset.csv")

In [58]:
df['Module_name'] = df['Filename'].apply(lambda x:x.split('/')[-1])

In [59]:
target = list(df.columns)[7]

In [60]:
df['Comment Text']

0      ''' || Functions used by more than one algorit...
1                                                  !TODO
2      !TODO: An empty dictionary would actually also...
3      !TODO: In some cases; US is not enough to hold...
4      # Hack to center SVG as justify-content is bro...
                             ...                        
851                                            XXX: Ugly
852                    XXX: unify Series\/Array as Array
853    XXX: using .values to check date type since Da...
854    XXX: We re-cache now just to make sure all ite...
855                           XXX: where should this be?
Name: Comment Text, Length: 856, dtype: object

In [7]:
links = df.groupby('Repo Name')['Comment-Introducing Revision'].agg([('count', 'count'), ('Comment-Introducing Revision', ', '.join)]).reset_index()

In [8]:
files = df.groupby('Repo Name')['Filename'].agg([('count', 'count'), ('Module_name', ', '.join)]).reset_index()

In [9]:
comments = df.groupby('Repo Name')['Comment Text'].agg([('count', 'count'), ('Comment Text', ', '.join)]).reset_index()

In [10]:
files.dtypes

Repo Name      object
count           int64
Module_name    object
dtype: object

In [11]:
links.dtypes

Repo Name                       object
count                            int64
Comment-Introducing Revision    object
dtype: object

In [12]:
final_df = files.merge(links[['Repo Name', 'Comment-Introducing Revision']], on='Repo Name', how='inner').merge(comments[['Repo Name', 'Comment Text']], on='Repo Name', how='inner')

In [13]:
final_df

Repo Name  count  \
0               AKSHAYUBHAT/DeepVideoAnalytics      4   
1                         Accenture/AmpliGraph      1   
2                         Aifred-Health/Vulcan      9   
3    AllenCellModeling/pytorch_integrated_cell      1   
4     AllenInstitute/mouse_connectivity_models      2   
..                                         ...    ...   
309                       yuanming-hu/exposure      1   
310                              yzhao062/pyod      3   
311                                 yzou2/CBST      1   
312                        zsdonghao/im2txt2im      1   
313                                  zvtvz/zvt      2   

                                           Module_name  \
0    repos/slim/nets/resnet_v2.py, repos/object_det...   
1                 ampligraph/latent_features/models.py   
2    vulcanai/models/metrics.py, vulcanai2/tests/mo...   
3                   data_providers/DataProvider3Dh5.py   
4    voxel_model/regionalized_model.py, voxel_model...   
..                                                 ...   
309                                             net.py   
310  pyod/models/base.py, pyod/test/test_so_gaal.py...   
311  data/cityscapesscripts/annotation/cityscapesLa...   
312                              tensorlayer/layers.py   
313  zvt/recorders/eastmoney/common.py, zvt/reader/...   

                          Comment-Introducing Revision  \
0    https://github.com/AKSHAYUBHAT/DeepVideoAnalyt...   
1    https://github.com/Accenture/AmpliGraph/commit...   
2    https://github.com/Aifred-Health/Vulcan/commit...   
3    https://github.com/AllenCellModeling/pytorch_i...   
4    https://github.com/AllenInstitute/mouse_connec...   
..                                                 ...   
309  https://github.com/yuanming-hu/exposure/commit...   
310  https://github.com/yzhao062/pyod/commit/c2cc41...   
311  https://github.com/yzou2/CBST/commit/10ff3a8f9...   
312  https://github.com/zsdonghao/im2txt2im/commit/...   
313  https://github.com/zvtvz/zvt/commit/2e3a0d462b...   

                                          Comment Text  
0    This is needed because the pre-activation vari...  
1                     TODO: Remove corruption entities  
2    #TODO: improve the copying of parameters, in_d...  
3    TODO add checking to make sure number of keys ...  
4    NOTE: probably more efficient to sort then str...  
..                                                 ...  
309  TODO: deal with png and jpeg files better - th...  
310  TODO: clean up these initialization, TODO: fix...  
311                 self.clearChanges() #TODO perhaps?  
312                   # reverse bw; before concat TODO  
313  FIXME:the > case, FIXME:we suppose history dat...  

[314 rows x 5 columns]

In [55]:
final_df.loc[1, 'Comment-Introducing Revision']

'https://github.com/Accenture/AmpliGraph/commit/68324ec3acecff31f67828ffafc1f6fc1d0948ed'

In [56]:
final_df.loc[1, 'Module_name']

'ampligraph/latent_features/models.py'

In [57]:
final_df.loc[1, 'Comment Text']

'TODO: Remove corruption entities'

In [59]:
final_df.loc[1, 'Comment-Introducing Revision']

'https://github.com/Accenture/AmpliGraph/commit/68324ec3acecff31f67828ffafc1f6fc1d0948ed'

In [46]:
html = requests.get('https://github.com/Accenture/AmpliGraph/commit/68324ec3acecff31f67828ffafc1f6fc1d0948ed').text
soup = BeautifulSoup(html, 'html.parser')
container_tags = soup.find_all('div', class_='js-diff-progressive-container')

In [53]:
for tag in container_tags:
    print(tag.find_all("a"))

[<a class="Link--primary Truncate-text" href="#diff-42c68c0cc7c309af68d0781d9d62bed27a2d6c9d8bc9902db1bf85d159264b51" title="ampligraph/datasets/conve_adapter.py">ampligraph/datasets/conve_adapter.py</a>, <a class="pl-5 dropdown-item btn-link" data-ga-click="View file, click, location:files_changed_dropdown" href="/Accenture/AmpliGraph/blob/68324ec3acecff31f67828ffafc1f6fc1d0948ed/ampligraph/datasets/conve_adapter.py" rel="nofollow" role="menuitem">
   View file
</a>, <a class="Link--inTextBlock" href="https://github.co/hiddenchars" target="_blank">Learn more about bidirectional Unicode characters</a>, <a class="btn-sm btn" data-view-component="true" href="{{ revealButtonHref }}">    Show hidden characters
</a>, <a aria-label="Expand Up" class="js-expand directional-expander single-expander" data-left-range="1-15" data-right-range="1-15" data-url="/Accenture/AmpliGraph/blob_excerpt/a916f99efd59947e1003b859a588988cf732e553?diff=unified&amp;direction=up&amp;in_wiki_context=&amp;last_left

In [ ]:
count = 0

directory = {}
for idx, row in final_df.iterrows():
    directory[row['Repo Name']] = []
    filenames = row['Module_name'].split(',') 
    links = row['Comment-Introducing Revision'].split(',')
    for fp, lk in zip(filenames, links):
        fname = fp.strip();
        lname = lk.strip();
        if requests.get(lname).status_code != 404:
            directory[row['Repo Name']].append({fp.strip():lk.strip()})

In [46]:
#### Get all the links that dont give a 404

In [45]:
requests.get('https://github.com/CPJKU/madmom/commit/9f8ca10ade46714f91c2f288559677da1215a48b').text

'\n\n\n\n\n\n\n<!DOCTYPE html>\n<html lang="en" data-color-mode="auto" data-light-theme="light" data-dark-theme="dark"  data-a11y-animated-images="system" data-a11y-link-underlines="false">\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>\n  <link rel="preconnect" href="https://avatars.githubusercontent.com">\n\n  \n\n  <link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/light-8cafbcbd78f4.css" /><link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/dark-31dc14e38457.css" /><link data-color-theme="dark_dimmed" crossorigin="anonymo

In [ ]:
  {'madmom/utils/rnnlib/rnnlib.py': 'https://github.com/CPJKU/madmom/commit/9f8ca10ade46714f91c2f288559677da1215a48b'},


In [32]:
import os
os.getcwd()

'/Users/pnair/Downloads'

In [44]:
import git

# Clone the repository if needed
repo_path = os.getcwd() + '/AmpliGraph'
repo = git.Repo.clone_from('https://github.com/Accenture/AmpliGraph.git', repo_path)

# Access the repository
repo = git.Repo(repo_path)

# Get the specific commit
commit_sha = '68324ec3acecff31f67828ffafc1f6fc1d0948ed'
commit = repo.commit(commit_sha)

# Access the file of interest
file_path = 'ampligraph/latent_features/models.py'
file_contents = commit.tree[file_path].data_stream.read().decode('utf-8')

print(file_contents)

# Copyright 2019 The AmpliGraph Authors. All Rights Reserved.
#
# This file is Licensed under the Apache License, Version 2.0.
# A copy of the Licence is available in LICENCE, or at:
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
import numpy as np
import tensorflow as tf
from sklearn.utils import check_random_state
import abc
from tqdm import tqdm
import logging
from .loss_functions import LOSS_REGISTRY
from .regularizers import REGULARIZER_REGISTRY
from .optimizers import OPTIMIZER_REGISTRY, SGDOptimizer, DEFAULT_LR
from .initializers import INITIALIZER_REGISTRY, DEFAULT_XAVIER_IS_UNIFORM
from ..evaluation import generate_corruptions_for_fit, to_idx, create_mappings, generate_corruptions_for_eval, \
    hits_at_n_score, mrr_score
from ..datasets import AmpligraphDatasetAdapter, NumpyDatasetAdapter, ConvEDatasetAdapter
from functools import partial

MODEL_REGISTRY = {}

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

#########################################

In [39]:
import requests
from bs4 import BeautifulSoup

# GitHub URL for the commit diff
github_url = "https://github.com/Accenture/AmpliGraph/commit/68324ec3acecff31f67828ffafc1f6fc1d0948ed#diff-b40ae4558942a73c8a28a8f94f1b090d394508f177cadceb59cef5ee5eede6bc"

# Fetch the content of the URL
response = requests.get(github_url)
html_content = response.text

# Parse HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find lines with "+" sign (added lines)
added_lines = soup.find_all('td', class_='blob-code blob-code-deletion js-file-line')

# Find lines with "-" sign (removed lines)
removed_lines = soup.find_all('div', class_='blob-code-inner blob-code-marker js-code-nav-pass js-skip-tagsearch')

# Extract and print added lines
print("Added Lines:")
for line in added_lines:
    print(line.text.strip())

# Extract and print removed lines
print("\nRemoved Lines:")
for line in removed_lines:
    print(line.text.strip())

Added Lines:
"""Set the filter to be used while generating an evaluation batch.
print('Setting ConvE filter.')
self.filter_mapping = filter_adapter.create_output_mapping('filter')

def create_onehot_outputs(self, dataset_type='train', use_filter=False):
use_filter : bool indicating whether to use a filter to generate onehot outputs
print('Creating onehot outputs: {} [Filter: {}]'.format(dataset_type, use_filter))
msg = 'use_filter=True but filter has not been set.'
def create_output_mapping(self, dataset_type='train'):
print('Creating output mapping: {}'.format(dataset_type))

out_filter = self.output_onehot[dataset_type][(i * batch_size):((i + 1) * batch_size), :]
out_filter[:, out[:, 2]] = 0.0 # set positive index to 0 for filtering
yield out, out_filter
def get_next_batch_with_filter(self, batch_size=1, dataset_type="test"):
"""Generator that returns the next batch of data along with the filter.
# If onehot outputs haven't been created for this dataset_type
self.create_onehot_output

In [51]:
import requests
import difflib
from git import Repo


def fetch_before_after(commit_url, filename):

    result = {"Added_lines":[], "Removed_lines":[]}
    # GitHub commit URL
#     commit_url = "https://api.github.com/repos/Accenture/AmpliGraph/commits/68324ec3acecff31f67828ffafc1f6fc1d0948ed"

    # Fetch commit data using GitHub API
    response = requests.get(commit_url)
    commit_data = response.json()
    files_changed = commit_data['files']

    # Find the file you're interested in
#     target_file = 'ampligraph/latent_features/models.py'

    target_file = filename
    target_file_changes = None

    for file in files_changed:
        if file['filename'] == target_file:
            target_file_changes = file
            break

    if target_file_changes:
        additions = target_file_changes['additions']
        deletions = target_file_changes['deletions']
        patch = target_file_changes['patch']

        # Split the patch into lines
        patch_lines = patch.split('\n')

        result["Added_lines"] = [line[1:] for line in patch_lines if line.startswith('+ ')][1:]
        result["Removed_lines"] = [line[1:] for line in patch_lines if line.startswith('- ')][1:]
        
        return result
    else:
        raise None
        

#         # Print added lines
#         print("Added lines:")
#         for line in added_lines:
#             print(line)

#         # Print removed lines
#         print("\nRemoved lines:")
#         for line in removed_lines:
#             print(line)
    

Added lines:
                                                           self.batches_count,
            self.initializer = INITIALIZER_REGISTRY[initializer](self.initializer_params,
        """After model fitting, save all the trained parameters in trained_model_params in some order.
        The order would be useful for loading the model.
        """Load the model from trained params.
            This function also set's the evaluation mode to do lazy loading of variables based on the number of
        """Get the embeddings for entities.
        """Initialize parameters of the model.
            This function creates and initializes entity and relation embeddings (with size k).
            If the graph is large, then it loads only the required entity embeddings (max:batch_size*2)
        # the latter 2 variables are passed only for large graphs.
            insert_lookup_op = self.sparse_mappings.insert(self.unique_entities,
                                                           t

In [63]:
text = requests.get("https://github.com/Accenture/AmpliGraph/commit/68324ec3acecff31f67828ffafc1f6fc1d0948ed#diff-b40ae4558942a73c8a28a8f94f1b090d394508f177cadceb59cef5ee5eede6bc")

In [22]:
headers = {'Accept': 'application/json'};
response = requests.get(
    
    "https://github.com/Accenture/AmpliGraph/commit/68324ec3acecff31f67828ffafc1f6fc1d0948ed#diff-b40ae4558942a73c8a28a8f94f1b090d394508f177cadceb59cef5ee5eede6bc"


)

In [27]:
url = "https://api.github.com/Accenture/AmpliGraph/commit/68324ec3acecff31f67828ffafc1f6fc1d0948ed#diff-b40ae4558942a73c8a28a8f94f1b090d394508f177cadceb59cef5ee5eede6bc"

In [30]:
response = requests.get(url)
response.json()

{'message': 'Not Found', 'documentation_url': 'https://docs.github.com/rest'}

In [17]:
commit_data = response.json()
files_changed = commit_data['files']

# Find the file you're interested in
#     target_file = 'ampligraph/latent_features/models.py'

target_file = filename
target_file_changes = None

for file in files_changed:
    if file['filename'] == target_file:
        target_file_changes = file
        break

if target_file_changes:
    additions = target_file_changes['additions']
    deletions = target_file_changes['deletions']
    patch = target_file_changes['patch']

    # Split the patch into lines
    patch_lines = patch.split('\n')

    result["Added_lines"] = [line[1:] for line in patch_lines if line.startswith('+ ')][1:]
    result["Removed_lines"] = [line[1:] for line in patch_lines if line.startswith('- ')][1:]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [31]:
import requests
import difflib
from git import Repo

# GitHub commit URL
commit_url = "https://api.github.com/repos/Accenture/AmpliGraph/commits/68324ec3acecff31f67828ffafc1f6fc1d0948ed"

# Fetch commit data using GitHub API
response = requests.get(commit_url)
commit_data = response.json()
files_changed = commit_data['files']

# Find the file you're interested in
target_file = 'ampligraph/latent_features/models.py'
target_file_changes = None

for file in files_changed:
    if file['filename'] == target_file:
        target_file_changes = file
        break

if target_file_changes:
    additions = target_file_changes['additions']
    deletions = target_file_changes['deletions']
    patch = target_file_changes['patch']

    # Split the patch into lines
    patch_lines = patch.split('\n')

    added_lines = [line[1:] for line in patch_lines if line.startswith('+ ')][1:]
    removed_lines = [line[1:] for line in patch_lines if line.startswith('- ')][1:]

    # Print added lines
    print("Added lines:")
    for line in added_lines:
        print(line)

    # Print removed lines
    print("\nRemoved lines:")
    for line in removed_lines:
        print(line)
else:
    print(f"File '{target_file}' not found in the commit changes.")
    

Added lines:
                                                           self.batches_count,
            self.initializer = INITIALIZER_REGISTRY[initializer](self.initializer_params,
        """After model fitting, save all the trained parameters in trained_model_params in some order.
        The order would be useful for loading the model.
        """Load the model from trained params.
            This function also set's the evaluation mode to do lazy loading of variables based on the number of
        """Get the embeddings for entities.
        """Initialize parameters of the model.
            This function creates and initializes entity and relation embeddings (with size k).
            If the graph is large, then it loads only the required entity embeddings (max:batch_size*2)
        # the latter 2 variables are passed only for large graphs.
            insert_lookup_op = self.sparse_mappings.insert(self.unique_entities,
                                                           t

In [32]:
commit_url = "https://api.github.com/repos/Accenture/AmpliGraph/commits/68324ec3acecff31f67828ffafc1f6fc1d0948ed"

# Fetch commit data using GitHub API
response = requests.get(commit_url)

commit_data = response.json()

In [33]:
req_file_name = 'ampligraph/latent_features/models.py'
for fp in commit_data['files']:
    if fp['filename'] == req_file_name:
        patch = fp['patch']

In [97]:
'\t'.join([line for line in patch.split('\n') if line.startswith('+') or line.startswith('-')])

'-    \t+\t-    \t-    \t+\t+\t-            \t+\t-        \t+\t-        \t+\t-            self.optimizer = OPTIMIZER_REGISTRY[optimizer](self.optimizer_params, \t-                                                           self.batches_count, \t+            self.optimizer = OPTIMIZER_REGISTRY[optimizer](self.optimizer_params,\t+                                                           self.batches_count,\t-        \t+\t-            self.initializer = INITIALIZER_REGISTRY[initializer](self.initializer_params, \t+            self.initializer = INITIALIZER_REGISTRY[initializer](self.initializer_params,\t-        \t+\t-        """After model fitting, save all the trained parameters in trained_model_params in some order. \t-        The order would be useful for loading the model. \t+        """After model fitting, save all the trained parameters in trained_model_params in some order.\t+        The order would be useful for loading the model.\t-        """Load the model from trained params. 

In [34]:
code = ""
for line in patch.split('\n'):
    if (line.startswith('+') or line.startswith('-')):
        code += line

In [101]:
apps = pd.read_csv('13-apps.csv', header=None)

In [119]:
apps["before"] = "https://github.com/"+apps[1]+"/commit/"+apps[3]
apps["after"] = "https://github.com/"+apps[1]+"/commit/"+apps[4]

In [130]:
apps.loc[7, ["before", "after"]].to_dict()

{'before': 'https://github.com/ealcobaca/pymfe/commit/70e58ce4494a521fabb30b96657629b9542b63f1',
 'after': 'https://github.com/ealcobaca/pymfe/commit/2d382ec699f34b2b76c1b4be235f4fab426ba594'}

In [113]:
count = 0
for link in a.tolist():
    print(requests.get(link).text)
    break
#         print(count)
#         count += 1

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


KeyboardInterrupt: 

In [ ]:
count==len(a)

In [111]:
count

0

In [137]:
round_1 = pd.read_excel('/Users/pnair/Documents/23Shades-master/labeling_process/Agreed Labels/round1_agreement.xlsx')
round_2 = pd.read_excel('/Users/pnair/Documents/23Shades-master/labeling_process/Agreed Labels/round2_agreement.xlsx')
round_3 = pd.read_excel('/Users/pnair/Documents/23Shades-master/labeling_process/Agreed Labels/round3_agreement.xlsx')
round_4 = pd.read_excel('/Users/pnair/Documents/23Shades-master/labeling_process/Agreed Labels/round4_agreement.xlsx')

In [140]:
print(round_1.shape)

(1114, 12)


In [143]:
print(round_2.shape)

(1114, 13)


In [142]:
print(round_3.shape)

(1114, 13)


In [141]:
print(round_4.shape)

(1114, 13)


In [148]:
round_4['MLTD?'].value_counts()

Yes    134
No      47
Name: MLTD?, dtype: int64

In [150]:
round_3['MLTD?'].value_counts()

Yes    93
No     24
Name: MLTD?, dtype: int64

In [151]:
round_2['MLTD?'].value_counts()

Yes    67
No     20
Name: MLTD?, dtype: int64

In [152]:
round_1['MLTD?'].value_counts()

Yes    42
No     12
Name: MLTD?, dtype: int64

In [153]:
round_4

Dataset ID    Repo Type                  Repo Name  \
0     aabhgedej  Application             vecto-ai/vecto   
1     aagcjdihh  Application               aleju/imgaug   
2     aadehgigf  Application     TheMTank/ai-distillery   
3     aacccifcd  Application       brendanhasz/probflow   
4     aagjaijdg  Application   python-adaptive/adaptive   
...         ...          ...                        ...   
1109  aaibfidii         Tool            microsoft/MMdnn   
1110  aahghgfic         Tool             biolab/orange3   
1111  aagadhddg         Tool  scikit-learn/scikit-learn   
1112  aagahedfi         Tool      gunthercox/ChatterBot   
1113  aahceahgh         Tool              kengz/SLM-Lab   

                                        Filename  \
0                               vecto/data/io.py   
1                               imgaug/imgaug.py   
2                            create_ICLR_task.py   
3                           src/probflow/core.py   
4                  adaptive/learner/learner2D.py   
...                                          ...   
1109           tests/test_conversion_imagenet.py   
1110                     Orange/data/variable.py   
1111  sklearn/gaussian_process/tests/test_gpr.py   
1112           chatterbot/controllers/storage.py   
1113                    slm_lab/agent/net/mlp.py   

                           Comment-Introducing Revision  \
0     https://github.com/vecto-ai/vecto/commit/9d8c4...   
1     https://github.com/aleju/imgaug/commit/6340236...   
2     https://github.com/TheMTank/ai-distillery/comm...   
3     https://github.com/brendanhasz/probflow/commit...   
4     https://github.com/python-adaptive/adaptive/co...   
...                                                 ...   
1109  https://github.com/microsoft/MMdnn/commit/fd11...   
1110  https://github.com/biolab/orange3/commit/a63c4...   
1111  https://github.com/scikit-learn/scikit-learn/c...   
1112  https://github.com/gunthercox/ChatterBot/commi...   
1113  https://github.com/kengz/SLM-Lab/commit/0ac2b3...   

                              Comment-Removing Revision  \
0                                          STILL_EXISTS   
1                                          STILL_EXISTS   
2                                          STILL_EXISTS   
3                                          STILL_EXISTS   
4                                          STILL_EXISTS   
...                                                 ...   
1109  https://github.com/microsoft/MMdnn/commit/73bd...   
1110  https://github.com/biolab/orange3/commit/61b75...   
1111  https://github.com/scikit-learn/scikit-learn/c...   
1112  https://github.com/gunthercox/ChatterBot/commi...   
1113  https://github.com/kengz/SLM-Lab/commit/c799a6...   

                                    Direct Link to File  \
0     https://github.com/vecto-ai/vecto/blob/9d8c432...   
1     https://github.com/aleju/imgaug/blob/6340236eb...   
2     https://github.com/TheMTank/ai-distillery/blob...   
3     https://github.com/brendanhasz/probflow/blob/7...   
4     https://github.com/python-adaptive/adaptive/bl...   
...                                                 ...   
1109  https://github.com/microsoft/MMdnn/blob/fd1112...   
1110  https://github.com/biolab/orange3/blob/a63c441...   
1111  https://github.com/scikit-learn/scikit-learn/b...   
1112  https://github.com/gunthercox/ChatterBot/blob/...   
1113  https://github.com/kengz/SLM-Lab/blob/0ac2b33e...   

                                           Comment Text MLTD? MLTD Broad Type  \
0                      TODO: move this to corpus module   Yes          Design   
1                            TODO rename sizes to size?   Yes            Code   
2                     todo try: \" \".join(foo.split())   Yes            Code   
3     TODO: x;y input should be able to be np arrays...   Yes          Design   
4     The while loop is needed because `stack_till` ...    No             NaN   
...                                                 ...   

In [3]:
import openai

openai.api_key = ""  # Replace with your actual API key

def check_satd_in_code(comment, new_code):
#     prompt = f"Given the following SATD comment and code:\n\nComment: {comment}\nCode: {new_code}\n\nDoes the new code contain Self-Admitted Technical Debt (SATD)?"
    prompt = "Help me write the code to add two numbers"
    response = openai.Completion.create(
        engine="davinci",
        prompt=prompt,
        max_tokens=1000, )

    answer = response.choices[0].text.strip()
    return answer

# Example SATD comment and new code
# satd_comment = final_df.loc[2,:]['Comment Text']
# new_code = code

result = check_satd_in_code("Hi ", "How are you")
print("Does the new code contain SATD?", result)

AuthenticationError: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.

In [160]:
final_df

Repo Name  count  \
0               AKSHAYUBHAT/DeepVideoAnalytics      4   
1                         Accenture/AmpliGraph      1   
2                         Aifred-Health/Vulcan      9   
3    AllenCellModeling/pytorch_integrated_cell      1   
4     AllenInstitute/mouse_connectivity_models      2   
..                                         ...    ...   
309                       yuanming-hu/exposure      1   
310                              yzhao062/pyod      3   
311                                 yzou2/CBST      1   
312                        zsdonghao/im2txt2im      1   
313                                  zvtvz/zvt      2   

                                           Module_name  \
0    repos/slim/nets/resnet_v2.py, repos/object_det...   
1                 ampligraph/latent_features/models.py   
2    vulcanai/models/metrics.py, vulcanai2/tests/mo...   
3                   data_providers/DataProvider3Dh5.py   
4    voxel_model/regionalized_model.py, voxel_model...   
..                                                 ...   
309                                             net.py   
310  pyod/models/base.py, pyod/test/test_so_gaal.py...   
311  data/cityscapesscripts/annotation/cityscapesLa...   
312                              tensorlayer/layers.py   
313  zvt/recorders/eastmoney/common.py, zvt/reader/...   

                          Comment-Introducing Revision  \
0    https://github.com/AKSHAYUBHAT/DeepVideoAnalyt...   
1    https://github.com/Accenture/AmpliGraph/commit...   
2    https://github.com/Aifred-Health/Vulcan/commit...   
3    https://github.com/AllenCellModeling/pytorch_i...   
4    https://github.com/AllenInstitute/mouse_connec...   
..                                                 ...   
309  https://github.com/yuanming-hu/exposure/commit...   
310  https://github.com/yzhao062/pyod/commit/c2cc41...   
311  https://github.com/yzou2/CBST/commit/10ff3a8f9...   
312  https://github.com/zsdonghao/im2txt2im/commit/...   
313  https://github.com/zvtvz/zvt/commit/2e3a0d462b...   

                                          Comment Text  
0    This is needed because the pre-activation vari...  
1                     TODO: Remove corruption entities  
2    #TODO: improve the copying of parameters, in_d...  
3    TODO add checking to make sure number of keys ...  
4    NOTE: probably more efficient to sort then str...  
..                                                 ...  
309  TODO: deal with png and jpeg files better - th...  
310  TODO: clean up these initialization, TODO: fix...  
311                 self.clearChanges() #TODO perhaps?  
312                   # reverse bw; before concat TODO  
313  FIXME:the > case, FIXME:we suppose history dat...  

[314 rows x 5 columns]

In [69]:
import pandas as pd
df = pd.read_excel('/Users/pnair/Documents/23Shades-master/labeling_process/Agreed Labels/round4_agreement.xlsx')
df.loc[0, 'Filename']

'vecto/data/io.py'

In [70]:
commit_data = requests.get('https://api.github.com/repos/vecto-ai/vecto/commits/9d8c43278ecd0a09edf5f89d88b3ff887123a9d9').json()

In [68]:
df.head()

Dataset ID    Repo Type      Repo Name  \
0  aahiieiig         Tool  kengz/SLM-Lab   
1  aaehidhih         Tool      cltk/cltk   
2  aaejjbhji  Application     loli/medpy   
3  aaejjacgf  Application     loli/medpy   
4  aaebbecaa  Application   Calysto/conx   

                                            Filename  \
0          slm_lab/agent/algorithm/algorithm_util.py   
1  cltk/parse/classical_latin/parse_latin_analyse...   
2                     src/medpy/graphcut/generate.py   
3    src/medpy/itkvtk/application/batch_watershed.py   
4                                    conx/widgets.py   

                        Comment-Introducing Revision  \
0  https://github.com/kengz/SLM-Lab/commit/4df110...   
1  https://github.com/cltk/cltk/commit/426111e168...   
2  https://github.com/loli/medpy/commit/183b17ee9...   
3  https://github.com/loli/medpy/commit/63d0245d4...   
4  https://github.com/Calysto/conx/commit/83f8cf6...   

  Comment-Removing Revision  \
0              STILL_EXISTS   
1              STILL_EXISTS   
2              STILL_EXISTS   
3              STILL_EXISTS   
4              STILL_EXISTS   

                                        Comment Text Is TD? Software TD Type  \
0  ''' || Functions used by more than one algorit...    Yes           Design   
1                                              !TODO    Yes             Code   
2  !TODO: An empty dictionary would actually also...    Yes      Requirement   
3  !TODO: In some cases; US is not enough to hold...    Yes           Defect   
4  # Hack to center SVG as justify-content is bro...    Yes           Defect   

  ML TD Type      ML TD Type 2 ML Pipeline Stage              Module_name  
0        NaN               NaN          Modeling        algorithm_util.py  
1        MCC  DS Configuration          Data Pre  parse_latin_analyses.py  
2       PODT  DS Configuration          Data Pre              generate.py  
3     MLKnow               NaN          Data Pre       batch_watershed.py  
4        NaN               NaN             Other               widgets.py

In [71]:
req_file_name = 'vecto/data/io.py'
for fp in commit_data['files']:
    if fp['filename'] == req_file_name:
        patch = fp['patch']

patch.split('\n')

In [81]:
df_not_null = df[df['MLTD?'].isnull()==False]

for idx, row in df_not_null.iterrows():
    df_not_null.loc[idx, 'Comment-Introducing Revision'] = row['Comment-Introducing Revision'].replace('https://github.com/', 'https://api.github.com/repos/').replace('/commit/', '/commits/')

/var/folders/3q/dw048hjx6d775m1t6pn8lf5h0000gs/T/ipykernel_14587/1534176503.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_not_null.loc[idx, 'Comment-Introducing Revision'] = row['Comment-Introducing Revision'].replace('https://github.com/', 'https://api.github.com/repos/').replace('/commit/', '/commits/')


In [82]:
for idx, row in df_not_null.iterrows():
    commit_data = requests.get(row['Comment-Introducing Revision']).json()
    
    filename = row['Filename']
    
    for fp in commit_data['files']:
        if fp['filename'] == req_file_name:
            patch = fp['patch']

Dataset ID    Repo Type                 Repo Name  \
0    aabhgedej  Application            vecto-ai/vecto   
1    aagcjdihh  Application              aleju/imgaug   
2    aadehgigf  Application    TheMTank/ai-distillery   
3    aacccifcd  Application      brendanhasz/probflow   
4    aagjaijdg  Application  python-adaptive/adaptive   
..         ...          ...                       ...   
176  aadccifca  Application             dtrckd/pymake   
177  aaadfagfh  Application            IntelAI/models   
178  aagjcddjh  Application   pyannote/pyannote-audio   
179  aagjcecjb  Application   pyannote/pyannote-audio   
180  aahgcbfhi  Application         deepchem/deepchem   

                                              Filename  \
0                                     vecto/data/io.py   
1                                     imgaug/imgaug.py   
2                                  create_ICLR_task.py   
3                                 src/probflow/core.py   
4                        adaptive/learner/learner2D.py   
..                                                 ...   
176                             pymake/expe/gramexp.py   
177  models/language_translation/tensorflow/transfo...   
178  pyannote/audio/embedding/approaches/center_los...   
179                      pyannote/audio/train/model.py   
180   deepchem/feat/tests/test_contact_fingerprints.py   

                          Comment-Introducing Revision  \
0    https://api.github.com/repos/vecto-ai/vecto/co...   
1    https://api.github.com/repos/aleju/imgaug/comm...   
2    https://api.github.com/repos/TheMTank/ai-disti...   
3    https://api.github.com/repos/brendanhasz/probf...   
4    https://api.github.com/repos/python-adaptive/a...   
..                                                 ...   
176  https://api.github.com/repos/dtrckd/pymake/com...   
177  https://api.github.com/repos/IntelAI/models/co...   
178  https://api.github.com/repos/pyannote/pyannote...   
179  https://api.github.com/repos/pyannote/pyannote...   
180  https://api.github.com/repos/deepchem/deepchem...   

    Comment-Removing Revision  \
0                STILL_EXISTS   
1                STILL_EXISTS   
2                STILL_EXISTS   
3                STILL_EXISTS   
4                STILL_EXISTS   
..                        ...   
176              STILL_EXISTS   
177              STILL_EXISTS   
178              STILL_EXISTS   
179              STILL_EXISTS   
180              STILL_EXISTS   

                                   Direct Link to File  \
0    https://github.com/vecto-ai/vecto/blob/9d8c432...   
1    https://github.com/aleju/imgaug/blob/6340236eb...   
2    https://github.com/TheMTank/ai-distillery/blob...   
3    https://github.com/brendanhasz/probflow/blob/7...   
4    https://github.com/python-adaptive/adaptive/bl...   
..                                                 ...   
176  https://github.com/dtrckd/pymake/blob/89ec9faa...   
177  https://github.com/IntelAI/models/blob/d459e40...   
178  https://github.com/pyannote/pyannote-audio/blo...   
179  https://github.com/pyannote/pyannote-audio/blo...   
180  https://github.com/deepchem/deepchem/blob/6d00...   

                                          Comment Text MLTD? MLTD Broad Type  \
0                     TODO: move this to corpus module   Yes          Design   
1                           TODO rename sizes to size?   Yes            Code   
2                    todo try: \" \".join(foo.split())   Yes            Code   
3    TODO: x;y input should be able to be np arrays...   Yes          Design   
4    The while loop is needed because `stack_till` ...    No             NaN   
..                                                 ...   ...             ...   
176                 @todo : verify  forbidden keywords    No             NaN   
177                      TODO: verify if it is working   Yes             M&T   
178  TODO add support for other metrics (ex. euclid...   Yes     Requirement   
179                            

In [92]:
from collections import defaultdict

dd = defaultdict(list)

In [94]:
for idx, row in df_not_null.iterrows():
    link = row['Comment-Introducing Revision']
    print(requests.get(link).status_code)
    break;

200


In [100]:
df_not_null.iloc[0:int(df_not_null.shape[0]/2), :].to_csv('first_90.csv', index=False)

In [101]:
df_not_null.iloc[int(df_not_null.shape[0]/2):, :].to_csv('last_90.csv', index=False)

In [19]:
import pandas as pd
a = pd.read_csv('last_90.csv')
b = pd.read_csv('first_90.csv')
a.head()

Dataset ID    Repo Type                                     Repo Name  \
0  aabbehgci  Application                               estnltk/estnltk   
1  aahfdhhgj  Application                                jupyter/nbdime   
2  aaabejjih  Application  RobotLocomotion/pytorch-dense-correspondence   
3  aacacagbd  Application                     richemslie/galvanise_zero   
4  aaededbjc  Application                              Erotemic/netharn   

                                            Filename  \
0  estnltk/neural_morph/new_neural_morph/seq2seq/...   
1                     nbmerge/diff/diff_notebooks.py   
2  modules/dense_correspondence_manipulation/coco...   
3                  src/ggpzero/distributed/server.py   
4                             netharn/hyperparams.py   

                        Comment-Introducing Revision  \
0  https://api.github.com/repos/estnltk/estnltk/c...   
1  https://api.github.com/repos/jupyter/nbdime/co...   
2  https://api.github.com/repos/RobotLocomotion/p...   
3  https://api.github.com/repos/richemslie/galvan...   
4  https://api.github.com/repos/Erotemic/netharn/...   

  Comment-Removing Revision  \
0              STILL_EXISTS   
1              STILL_EXISTS   
2              STILL_EXISTS   
3              STILL_EXISTS   
4              STILL_EXISTS   

                                 Direct Link to File  \
0  https://github.com/estnltk/estnltk/blob/4ac613...   
1  https://github.com/jupyter/nbdime/blob/518273b...   
2  https://github.com/RobotLocomotion/pytorch-den...   
3  https://github.com/richemslie/galvanise_zero/b...   
4  https://github.com/Erotemic/netharn/blob/af0df...   

                                        Comment Text MLTD? MLTD Broad Type  \
0  checked; optimal [1; 2]. TODO: test with decay...   Yes             M&T   
1  \"\"\"Tools for diffing notebooks. ||  || All ...   Yes            Docu   
2  this is needed because the mask is no longer j...    No             NaN   
3  '' ||  || XXX write to a temporary file before...   Yes     Requirement   
4                            TODO: software versions   Yes            Docu   

  MLTD Specific Type 1 MLTD Specific Type 2 ML Pipeline Stage  
0              HP Conf                  NaN          Training  
1                  NaN                  NaN          Data Acq  
2                  NaN                  NaN               NaN  
3     DS Configuration                  NaN          Data Acq  
4     DS Configuration                  NaN          Training

In [46]:
import requests
import difflib
from git import Repo

def extract_code(commit_url, target_file):
    response = None;
    # GitHub commit URL
    #commit_url = "https://api.github.com/repos/Accenture/AmpliGraph/commits/68324ec3acecff31f67828ffafc1f6fc1d0948ed"

    # Fetch commit data using GitHub API
    response = requests.get(commit_url)
    commit_data = response.json()
    print(commit_data.keys());
    
    if 'files' in commit_data.keys():
        print('We haave a files key');
        files_changed = commit_data['files']

        target_file_changes = None

        for file in files_changed:
            if file['filename'] == target_file:
                target_file_changes = file
                break
        
                
        if target_file_changes and 'patch' in target_file_changes.keys():
            print('We have a patch as well');
            response = target_file_changes['patch'];
    return response;

In [34]:
a['Code'] = ''

In [62]:
from tqdm import tqdm;
for idx, row in tqdm(a.iloc[60:, :].iterrows()):
    commit_url = row['Comment-Introducing Revision']
    target_file = row['Filename']
    response = extract_code(commit_url, target_file)
    if response is not None:
        a.loc[idx, 'Code'] = str(response)

1it [00:02,  2.06s/it]

dict_keys(['sha', 'node_id', 'commit', 'url', 'html_url', 'comments_url', 'author', 'committer', 'parents', 'stats', 'files'])
We haave a files key
We have a patch as well


2it [00:04,  2.25s/it]

dict_keys(['sha', 'node_id', 'commit', 'url', 'html_url', 'comments_url', 'author', 'committer', 'parents', 'stats', 'files'])
We haave a files key
We have a patch as well


3it [00:05,  1.86s/it]

...

dict_keys(['sha', 'node_id', 'commit', 'url', 'html_url', 'comments_url', 'author', 'committer', 'parents', 'stats', 'files'])
We haave a files key
We have a patch as well


30it [00:22,  1.41s/it]

dict_keys(['sha', 'node_id', 'commit', 'url', 'html_url', 'comments_url', 'author', 'committer', 'parents', 'stats', 'files'])
We haave a files key
We have a patch as well


31it [00:23,  1.31it/s]

dict_keys(['sha', 'node_id', 'commit', 'url', 'html_url', 'comments_url', 'author', 'committer', 'parents', 'stats', 'files'])
We haave a files key
We have a patch as well


In [ ]:
a.

In [80]:
a.to_csv('Capstone_code_data_1.csv', index=False)

In [77]:
from tqdm import tqdm;
for idx in tqdm(range(40, 90)):
    commit_url = b.loc[idx, 'Comment-Introducing Revision']
    target_file = b.loc[idx, 'Filename']
    response = extract_code(commit_url, target_file)
    if response is not None:
        b.loc[idx, 'Code'] = str(response)

  2%|██▋                                                                                                                                      | 1/50 [00:00<00:21,  2.25it/s]

dict_keys(['sha', 'node_id', 'commit', 'url', 'html_url', 'comments_url', 'author', 'committer', 'parents', 'stats', 'files'])
We haave a files key
We have a patch as well


  4%|█████▍                                                                                                                                   | 2/50 [00:00<00:21,  2.26it/s]

dict_keys(['sha', 'node_id', 'commit', 'url', 'html_url', 'comments_url', 'author', 'committer', 'parents', 'stats', 'files'])
We haave a files key
We have a patch as well


  6%|████████▏                                                                                                                                | 3/50 [00:01<00:16,  2.91it/s]

...

dict_keys(['sha', 'node_id', 'commit', 'url', 'html_url', 'comments_url', 'author', 'committer', 'parents', 'stats', 'files'])
We haave a files key
We have a patch as well


 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎  | 49/50 [00:24<00:00,  1.73it/s]

dict_keys(['sha', 'node_id', 'commit', 'url', 'html_url', 'comments_url', 'author', 'committer', 'parents', 'stats', 'files'])
We haave a files key
We have a patch as well


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:24<00:00,  2.05it/s]

dict_keys(['sha', 'node_id', 'commit', 'url', 'html_url', 'comments_url', 'author', 'committer', 'parents', 'stats', 'files'])
We haave a files key
We have a patch as well


In [81]:
b.iloc[:, :].to_csv('Capstone_code_data_2.csv', index=False)

In [2]:
import pandas as pd
a = pd.read_csv('Capstone_code_data_1.csv')
b = pd.read_csv('Capstone_code_data_2.csv')
final_df = pd.concat([a, b], ignore_index=True)

In [91]:
final_df = pd.concat([a, b], ignore_index=True)

In [105]:
[i.lstrip('+') for i in final_df['Code'][0].split('\n')[1:]]

['import os',
 'from ..common_config import *',
 '',
 '# training',
 'l2_loss_weight = 0  # 0.0005',
 'batch_size = 5  # checked',
 'lr_method = "sgd"  # sgd | adam | adagrad | rmsprop | momentum',
 'lr = 1.0  # checked, optimal [1, 2]. TODO: test with decay. Run lr=1 for > 100 iterations.',
 'lr_decay = 1.',
 'lr_decay_strategy = None  # "on-no-improvement" | "step" | "exponential" | None',
 '',
 'nepoch_no_imprv = 50',
 'max_epochs = 400',
 '',
 '# decoder',
 'dim_tag = 150  # checked',
 'tag_embeddings_dropout = 0.5',
 'decoder_maximum_iterations = 12  # find maximum from the training corpus',
 'attention = None  # None | analysis-tag | analysis-category | char | sentence . Set to None to disable attention',
 'attention_mechanism = "luong"  # bahdanau | luong',
 'trainer = "basic"  # scheduled | basic',
 'scheduled_trainer_sampling_prob = .2',
 'use_crf = False  # if crf, training is 1.7x slower on CPU',
 '',
 '# evaluation',
 'eval_batch_size = 200']

In [106]:
final_df.columns

Index(['Dataset ID', 'Repo Type', 'Repo Name', 'Filename',
       'Comment-Introducing Revision', 'Comment-Removing Revision',
       'Direct Link to File', 'Comment Text', 'MLTD?', 'MLTD Broad Type',
       'MLTD Specific Type 1', 'MLTD Specific Type 2', 'ML Pipeline Stage',
       'Code'],
      dtype='object')

In [108]:
final_df['MLTD Broad Type'].value_counts()

Requirement    37
Code           19
M&T            17
Design         15
Defect          9
Docu            6
Conf            4
Name: MLTD Broad Type, dtype: int64

In [109]:
len([1, 2, 3, 1]), len(set([1, 2, 3, 1]))

(4, 3)

In [112]:
sorted('qwerty') == ['e', 'q', 'r', 't', 'w', 'y']

True

In [ ]:
class Solution:
    def groupAnagrams(self, strs: List[str]) -> List[List[str]]:
        res = defaultdict(list)
        
        for s in strs:
            

In [113]:
[1, 3, 2, 1] + [1, 3, 2, 1]

[1, 3, 2, 1, 1, 3, 2, 1]

In [1]:
final_df

NameError: name 'final_df' is not defined

In [10]:
df.shape

(856, 12)